In [9]:
import requests
import configparser
import snowflake.connector
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load configuration
config = configparser.ConfigParser()
config.read("config.ini")

SNOWFLAKE_CONFIG = {
    "user": config["snowflake"]["user"],
    "password": config["snowflake"]["password"],
    "account": config["snowflake"]["account"],
    "warehouse": config["snowflake"]["warehouse"],
    "database": config["snowflake"]["database"],
    "schema": config["snowflake"]["schema"],
}


# Hugging Face API configuration for Mistral v2
API_URL = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.2"
API_KEY = config["huggingface"]["api_key"]


# Load the model with authentication (if required)
embedding_model = SentenceTransformer('all-MiniLM-L6-v2', use_auth_token=API_KEY)

HEADERS = {"Authorization": f"Bearer {API_KEY}", "Content-Type": "application/json"}

c:\Users\Ananya.Mehta\AppData\Local\Programs\Python\Python311\Lib\site-packages\sentence_transformers\SentenceTransformer.py:196: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v4 of SentenceTransformers.
  warnings.warn(


In [10]:
# Step 1: Create AI Solutions Table in Snowflake
def create_ai_solutions_table():
    conn = snowflake.connector.connect(**SNOWFLAKE_CONFIG)
    cursor = conn.cursor()
    
    query = """
    CREATE TABLE IF NOT EXISTS AI_SOLUTIONS (
        INCIDENT_ID STRING PRIMARY KEY,
        SUBJECT STRING,
        IMPACT_LEVEL STRING,
        AFFECTED_SERVICE STRING,
        TIMESTAMP TIMESTAMP,
        DESCRIPTION STRING,
        AI_GENERATED_SOLUTION STRING,
        ROOT_CAUSE_ANALYSIS STRING
    );
    """
    cursor.execute(query)
    conn.commit()
    cursor.close()
    conn.close()
    
    print("AI_SOLUTIONS table checked/created successfully.")

In [11]:
# Step 2: Fetch Latest Email Alert from Snowflake
def fetch_latest_email_alert():
    conn = snowflake.connector.connect(**SNOWFLAKE_CONFIG)
    cursor = conn.cursor()
    
    query = """
        SELECT subject, INCIDENT_ID, impact_level, affected_service, timestamp, description
        FROM cleaned_alerts
        ORDER BY timestamp DESC
        LIMIT 1;
    """
    cursor.execute(query)
    latest_alert = cursor.fetchone()
    column_names = [desc[0].lower() for desc in cursor.description]
    cursor.close()
    conn.close()
    
    return dict(zip(column_names, latest_alert)) if latest_alert else None

In [12]:
def generate_root_cause_analysis(alert_data, similar_solution=None):
    if not alert_data:
        return "No new alerts found."
    
    formatted_alert = f"""
    Incident ID: {alert_data['incident_id']}
    Subject: {alert_data['subject']}
    Impact Level: {alert_data['impact_level']}
    Affected Service: {alert_data['affected_service']}
    Timestamp: {alert_data['timestamp']}
    Description: {alert_data['description']}
    """
    
    if similar_solution:
        prompt = f"""
        Analyze this alert and its similar solution to provide a detailed root cause analysis:\n\n{formatted_alert}\n\nSimilar Solution:\n{similar_solution}
        """
    else:
        prompt = f"""
        Analyze this alert and provide a detailed root cause analysis:\n\n{formatted_alert}
        """
    
    data = {"inputs": prompt}
    
    try:
        response = requests.post(API_URL, headers=HEADERS, json=data)
        response.raise_for_status()
        result = response.json()
        
        if isinstance(result, list) and result:
            rca_details = result[0].get("generated_text", "Root cause analysis not found.")
            print("Root Cause Analysis Details:")
            print(rca_details)
            return rca_details
        
        print("Root cause analysis not found.")
        return "Root cause analysis not found."
    
    except Exception as e:
        print(f"Error: {e}")
        return "API request failed."


In [13]:
def store_ai_solution(alert_data, ai_solution, root_cause_analysis):
    conn = snowflake.connector.connect(**SNOWFLAKE_CONFIG)
    cursor = conn.cursor()
    query = """
        INSERT INTO AI_SOLUTIONS (number, incident_subject, priority, business_service, opened, short_description, ai_generated_Solution)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    """
    
    # Assuming alert_data contains necessary fields or you need to map them appropriately
    values = (
        alert_data.get('incident_id', None),  # Map to 'number' if necessary
        alert_data.get('subject', None),
        alert_data.get('impact_level', None),  # Map to 'priority' if necessary
        alert_data.get('affected_service', None),  # Map to 'business_service' if necessary
        alert_data.get('timestamp', None),  # Map to 'opened' if necessary
        alert_data.get('description', None),  # Map to 'short_description' if necessary
        ai_solution
    )
    
    cursor = conn.cursor()
    cursor.execute(query, values)
    conn.commit()
    cursor.close()


In [14]:
# Step 5: Load Existing Embeddings from Snowflake
def load_existing_embeddings():
    conn = snowflake.connector.connect(**SNOWFLAKE_CONFIG)
    cursor = conn.cursor()
    query = """
        SELECT number, short_description, ai_generated_Solution
        FROM AI_SOLUTIONS;
    """
    cursor = conn.cursor()
    cursor.execute(query)
    rows = cursor.fetchall()
    cursor.close()

    incident_ids = [row[0] for row in rows]
    descriptions = [row[1] for row in rows]
    solutions = [row[2] for row in rows]

    return incident_ids, descriptions, solutions

In [15]:

# Step 6: Build FAISS Index and Perform Similarity Check
def build_faiss_index(descriptions):
   embeddings = embedding_model.encode(descriptions, convert_to_numpy=True)
   dim = embeddings.shape[1]
   index = faiss.IndexFlatL2(dim)
   index.add(embeddings)
   return index

def check_for_similar_alert(new_description, index, descriptions, incident_ids, solutions, threshold=0.85):
   new_embedding = embedding_model.encode([new_description], convert_to_numpy=True)
   D, I = index.search(new_embedding, k=1)
   similarity = 1 / (1 + D[0][0])  # Convert L2 to similarity
   
   if similarity >= threshold:
       match_idx = I[0][0]
       print(f"Similar alert found! Similarity: {similarity:.2f}")
       return solutions[match_idx]
   
   print(f"No similar alert found. Similarity: {similarity:.2f}")
   return None

In [16]:
# Main Workflow Execution
create_ai_solutions_table()

latest_alert = fetch_latest_email_alert()

if latest_alert:
   incident_ids, descriptions, solutions = load_existing_embeddings()

   if descriptions:
       index = build_faiss_index(descriptions)
       matched_solution = check_for_similar_alert(
           latest_alert["description"], index, descriptions, incident_ids, solutions
       )
   else:
       matched_solution = None

   if matched_solution:
       root_cause_analysis = generate_root_cause_analysis(latest_alert, matched_solution)
       store_ai_solution(latest_alert, matched_solution, root_cause_analysis)
   else:
       generated_solution = generate_root_cause_analysis(latest_alert)  # Generate RCA without similar solution
       store_ai_solution(latest_alert, generated_solution, generated_solution)  # Store both as fallback



AI_SOLUTIONS table checked/created successfully.
Similar alert found! Similarity: 1.00
Root Cause Analysis Details:

        Analyze this alert and its similar solution to provide a detailed root cause analysis:


    Incident ID: INC-20250407-002
    Subject: Unauthorized Database Access Attempt Detected
    Impact Level: High
    Affected Service: Database Cluster
    Timestamp: 2025-04-07 10:30:00
    Description: An unauthorized access attempt was detected on the primary database cluster. Multiple failed login attempts were recorded from an unrecognized IP address. Immediate action is required to secure the database and investigate the source of the attack.
    

Similar Solution:
API request failed.
         Error Message:
        Unauthorized
         Request detail:
         {
            "Operation": "login",
            "ClientIP": "192.168.1.10",
            "Username": "UA_Anonymous",
            "Password": "PWD_Anonymous"
         }
         Description: An attempt was mad